In [1]:
import os
import librosa
import pandas as pd
import numpy as np

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [2]:
data_folder = os.path.join(os.path.dirname(os.getcwd()), 'data', 'raw_file')

df = pd.DataFrame({'gwm': [], "mfccs":[], "spectogram":[],"zcr":[],"sc":[],"ss":[],"srf":[],"sf":[],"cv":[],"cd":[], 'emotion': []})

file_num = 1
for filename in os.listdir(data_folder):
    # GWM
    signal, sample_rate = librosa.load(os.path.join(data_folder, filename), sr=None, duration=2.5, offset=0.6)
    gwm = librosa.feature.rms(y=signal).flatten() 
    
    # EMOCJA
    file_parts = filename.split('-')
    emotion = int(file_parts[2])

    #MFCCS
    mfccs = librosa.feature.mfcc(y=np.squeeze(signal), sr=sample_rate).flatten() 

    #SPECTOGRAM
    spectogram=librosa.feature.melspectrogram(y=signal, sr=sample_rate)
    spectogram_DB = librosa.power_to_db(spectogram, ref=np.max).flatten() 
    
    #Zero Crossing Rate
    zcr = librosa.feature.zero_crossing_rate(signal).flatten()

    #Spectral centroid
    sc = librosa.feature.spectral_centroid(y=signal, sr=sample_rate).flatten()

    #Spectral Spread
    ss = librosa.feature.spectral_bandwidth(y=signal, sr=sample_rate).flatten()

    # Spectral Flux
    sf = librosa.onset.onset_strength(y=signal, sr=sample_rate).flatten()

    # Spectral Roll-Off
    srf = librosa.feature.spectral_rolloff(y=signal, sr=sample_rate).flatten()

    # Chroma Vector
    cv = librosa.feature.chroma_stft(y=signal, sr=sample_rate).flatten()

    #Chroma Deviation
    cv_mean = np.mean(cv, axis=1)  # Średnia dla każdego z 12 chroma
    cd = np.sqrt(np.mean((cv - cv_mean[:, np.newaxis])**2, axis=1))


    # Dodawanie do DataFrame
    temp = pd.DataFrame({'gwm': [gwm],"mfccs": [mfccs],"spectogram":[spectogram_DB],"zcr":[zcr],"sc":[sc],"ss":[ss],"sf":[sf],"srf":[srf],"cv":[cv],"cd":[cd], 'emotion': [emotion]})
    df = df._append(temp, ignore_index=True)
    
    #safety
    file_num += 1
    print(file_num)

AxisError: axis 1 is out of bounds for array of dimension 1

In [ ]:
df.to_csv(r"../data/data_proccesed",index=False)

In [3]:
def extract_features(data, sample_rate):
    # ZCR
    result = np.array([])
    zcr = np.mean(librosa.feature.zero_crossing_rate(y=data).T, axis=0)
    result=np.hstack((result, zcr))

    # Chroma_stft
    stft = np.abs(librosa.stft(data))
    chroma_stft = np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate).T, axis=0)
    result = np.hstack((result, chroma_stft))

    # MFCC
    mfcc = np.mean(librosa.feature.mfcc(y=data, sr=sample_rate).T, axis=0)
    result = np.hstack((result, mfcc))

    # Root Mean Square Value
    rms = np.mean(librosa.feature.rms(y=data).T, axis=0)
    result = np.hstack((result, rms))

    # MelSpectogram
    mel = np.mean(librosa.feature.melspectrogram(y=data, sr=sample_rate).T, axis=0)
    result = np.hstack((result, mel))
    
    return result

In [4]:
def get_features():
    data, sample_rate = librosa.load(os.path.join(data_folder, filename), duration=2.5, offset=0.6)
    
    res = extract_features(data, sample_rate)
    result = np.array(res)
    

    return result

In [5]:
data_folder = os.path.join(os.path.dirname(os.getcwd()), 'data', 'raw_file')
X, Y = [], []
for filename in os.listdir(data_folder):
    feature = get_features()
    file_parts = filename.split('-')
    emotion = int(file_parts[2])
    X.append(feature)
    Y.append(emotion)

In [6]:
Features = pd.DataFrame(X)
Features['labels'] = Y
Features.to_csv(r"../data/data_proccesed.csv",index=False)
Features.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,labels
0,0.321275,0.729664,0.750033,0.730624,0.735275,0.713529,0.660531,0.684966,0.733049,0.753972,0.736093,0.717019,0.668645,-645.880737,72.619637,0.841305,16.399446,10.241591,0.653901,-4.328002,-4.258932,-14.575824,-3.806285,1.022422,-4.042665,2.519602,-4.972247,0.566447,1.035527,-5.416451,-2.449469,-1.782306,-1.212806,0.002988,0.000002,0.000074,0.004795,0.027716,0.041546,0.023402,0.004569,0.024622,0.032647,0.027073,0.029055,0.014219,0.010516,0.009668,0.019981,0.048182,0.065960,0.017123,0.012617,0.024195,0.051398,0.018097,0.002698,0.005194,0.016427,0.024416,0.003669,0.000951,0.001133,0.001083,0.000416,0.000248,0.000263,0.000401,0.000833,0.001230,0.000834,0.000573,0.000285,0.000708,0.002412,0.001459,0.001920,0.004633,0.006113,0.002087,0.000741,0.000597,0.001077,0.000852,0.000529,0.001055,0.001177,0.000829,0.001363,0.001186,0.001123,0.000686,0.000578,0.000421,0.000330,0.001411,0.001208,0.000084,0.000081,0.000174,0.000128,0.000176,0.000145,0.000509,0.000866,0.000756,0.000831,0.000408,0.000250,0.000115,0.000127,0.000117,0.000127,0.000193,0.000231,0.000256,0.000356,0.000481,0.000406,0.000587,0.000153,0.000074,0.000026,0.000016,0.000020,0.000030,0.000051,0.000065,0.000052,0.000110,0.000103,0.000061,0.000079,0.000041,0.000030,0.000035,0.000049,0.000042,0.000027,0.000021,0.000014,0.000010,0.000008,0.000005,0.000005,0.000006,0.000006,8.748650e-06,0.000007,0.000004,0.000003,0.000004,0.000004,0.000004,0.000003,0.000002,0.000002,0.000005,0.000008,0.000007,0.000005,4.245834e-07,1
1,0.256113,0.621621,0.580278,0.598515,0.628296,0.598015,0.597272,0.650352,0.698635,0.681863,0.624018,0.605427,0.603519,-573.702881,74.766006,-16.835480,9.510839,-8.442755,-7.398400,-9.191018,-13.571815,-6.444541,-1.531597,-7.231005,-3.087767,-7.518957,0.035701,-9.967037,-3.094857,-3.932122,-2.832257,-1.936052,-0.699492,0.005244,0.000017,0.000074,0.000163,0.000112,0.000081,0.001295,0.015409,0.097839,0.192161,0.023570,0.012770,0.055444,0.025806,0.046543,0.122465,0.054262,0.017274,0.016355,0.006661,0.004392,0.007212,0.010751,0.028717,0.069067,0.027179,0.013818,0.019130,0.019606,0.097280,0.076587,0.029708,0.038095,0.028472,0.007926,0.004056,0.005038,0.005805,0.015003,0.020552,0.064104,0.029283,0.013830,0.011655,0.006716,0.013447,0.017863,0.024282,0.011514,0.009457,0.008353,0.005124,0.010570,0.013491,0.013449,0.010430,0.009864,0.006305,0.002741,0.002161,0.006513,0.014841,0.022626,0.030875,0.012503,0.004180,0.000773,0.000807,0.001940,0.001010,0.000690,0.000474,0.000528,0.002033,0.000809,0.000341,0.000599,0.000652,0.000896,0.000741,0.000581,0.001944,0.001701,0.001076,0.000718,0.001069,0.000367,0.000219,0.000369,0.000252,0.000198,0.000475,0.000209,0.000160,0.000584,0.000354,0.000411,0.000352,0.000244,0.000358,0.000155,0.000217,0.000089,0.000063,0.000063,0.000054,0.000074,0.000073,0.000066,0.000054,0.000055,0.000047,0.000016,0.000009,1.694765e-05,0.000035,0.000031,0.000031,0.000031,0.000022,0.000018,0.000016,0.000013,0.000012,0.000007,0.000008,0.000008,0.000005,5.802854e-07,1
2,0.207361,0.665823,0.670921,0.684690,0.707034,0.734071,0.654613,0.594455,0.626005,0.704741,0.667890,0.682175,0.696571,-555.065674,75.601807,-5.704805,13.053980,4.726385,-4.209593,-17.232430,-9.602064,-4.618643,-11.987761,-2.894989,-3.832055,-7.085772,-1.316404,-8.480334,-2.448134,-6.416794,-3.772164,-2.375920,-6.454505,0.005497,0.000037,0.000265,0.001084,0.003611,0.007876,0.064679,0.092985,0.042058,0.023150,0.013104,0.010007,0.019096,0.034062,0.025769,0.145445,0.380293,

In [7]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder

X = Features.iloc[: ,:-1].values
Y = Features['labels'].values

encoder = OneHotEncoder()
Y = encoder.fit_transform(np.array(Y).reshape(-1,1)).toarray()

x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

scaler = MinMaxScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)